<a href="https://colab.research.google.com/github/leedhn/papago/blob/main/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive #edit
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NAVER 

Mounted at /content/drive
/content/drive/MyDrive/NAVER


In [ ]:
import pandas as pd #edit
import numpy as np
from glob import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import random
import logging
import math
import torchtext
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
from torch import Tensor
import io
import time

In [ ]:
logger = logging.getLogger('my_logger')

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logger.info('This message has a date/time timestamp')


logger.propagate = False # do not pass logs to the default logger

# Create handlers
c_handler = logging.StreamHandler()
EPOCH=10
f_handler = logging.FileHandler(f'evaluate.log', mode='w')
c_handler.setLevel(logging.INFO)
f_handler.setLevel(logging.INFO)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(asctime)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)



2021-06-10 12:39:45,566 - This message has a date/time timestamp


In [ ]:
%cd data
txt_list = glob('*.txt') #edit
datas = {}
for txt in txt_list:
    datas[txt] = pd.read_csv(txt,header=None)
    name = txt[-10:-4]
    datas[txt].columns = [name]
    for i in range(len(datas[txt][name])):
        datas[txt][name][i] = np.fromstring(datas[txt][name][i] ,dtype=int,sep=' ').tolist()
        #datas[txt][name][i].append(1)
    logger.info(f'Load {txt} finished')
%cd ../

[Errno 2] No such file or directory: 'data'
/content/drive/My Drive/NAVER/data


2021-06-10 12:39:47,736 - Load train_source.txt finished
2021-06-10 12:39:48,740 - Load train_target.txt finished
2021-06-10 12:39:49,236 - Load test_target.txt finished
2021-06-10 12:39:49,641 - Load test_source.txt finished


/content/drive/My Drive/NAVER


In [66]:
#%cd trained_model
model_list = glob('trained_model/transformer*.pth')
encoder_list = glob('trained_model/encoder*.pth')
decoder_list = glob('trained_model/attn*.pth')
#%cd ../

#Data Processing

In [ ]:
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
PAD_IDX = 3#de_vocab['<pad>']
BOS_IDX = 2#de_vocab['<bos>']
EOS_IDX = 1#de_vocab['<eos>']
test_data_new = []


In [ ]:
# from torch.nn.utils.rnn import pad_sequence
# from torch.utils.data import DataLoader

# def generate_batch(data_batch):
#   de_batch, en_batch = [], []
#   for (de_item, en_item) in data_batch:
    
#     de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
#     en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
#   de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)#, batch_first=True)
#   en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)#, batch_first=True)

#   return de_batch, en_batch

In [ ]:

for iter in range(len(datas['test_source.txt'])):
    test_data_new.append((torch.Tensor(datas['test_source.txt']['source'][iter]).type(torch.long),torch.Tensor(datas['test_target.txt']['target'][iter]).type(torch.long)))

# test_iter_new = DataLoader(test_data_new, batch_size=BATCH_SIZE,
#                        shuffle=False, collate_fn=generate_batch)

#Model

In [ ]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
                
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + 
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
# SRC_VOCAB_SIZE = len(de_vocab)
# TGT_VOCAB_SIZE = len(en_vocab)

SRC_VOCAB_SIZE = 1000#len(de_vocab)
TGT_VOCAB_SIZE = 1000#len(en_vocab)

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 16#64#16

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, 
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

#Evaluate Transformers

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys
    
def translate(model, idx):
  model.eval()
  tokens = test_data_new[idx][0]
  num_tokens = len(tokens)
  src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()

  return tokens.tolist(),test_data_new[idx][1].tolist(),tgt_tokens[1:-1].tolist()

In [ ]:
def correct_num(model): #whole answer
  model.eval()
  corrects = 0
  for idx in range(len(test_data_new)):
    tokens = test_data_new[idx][0]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()

    if test_data_new[idx][1].tolist()==tgt_tokens[1:-1].tolist():
        corrects +=1
  return corrects

In [ ]:
def correct_num_wide(model): #sliced answer 
  model.eval()
  corrects = 0
  for idx in range(len(test_data_new)):
    tokens = test_data_new[idx][0]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()

    if test_data_new[idx][1].tolist()==tgt_tokens[1:len(test_data_new[idx][1])+1].tolist():
        corrects +=1
  return corrects

In [52]:
def correct_num_vector(model): #sliced answer 
  model.eval()
  corrects = 0
  accuracy = 0.0
  for idx in range(len(test_data_new)):
    tokens = test_data_new[idx][0]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()

    tgt = test_data_new[idx][1].to(device)

    for i in range(min(len(tgt),len(tgt_tokens)-1)):
        
        if tgt[i] == tgt_tokens[i+1]:
            corrects +=1
    cur_acc = corrects/max(len(tgt_tokens),len(tgt))
    accuracy+=cur_acc
    corrects = 0
  
  return accuracy/len(test_data_new)

In [53]:
for model_path in model_list:
    transformer = torch.load(model_path)
    transformer.eval()
    #acc_c = correct_num(transformer)
    #acc_cw = correct_num_wide(transformer)
    acc_v = correct_num_vector(transformer)
    print(model_path,"'s accuracy")
    #print("",acc_c/len(test_data_new))
    #print("",acc_cw/len(test_data_new))
    print("",acc_v)
    

trained_model/transformer.pth 's accuracy
 0.2251138583720375
trained_model/transformer_epoch32.pth 's accuracy
 0.29049130928871736
trained_model/transformer_no1.pth 's accuracy
 0.41821483201269405
trained_model/transformer_epoch_40.pth 's accuracy
 0.4965256591883321
trained_model/transformer_epoch_64.pth 's accuracy
 0.5112848484506195
trained_model/transformer_epoch_100.pth 's accuracy
 0.5025680825309998
trained_model/transformer_epoch_16.pth 's accuracy
 0.42968025859986925


#Evaluate GRU

In [59]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [62]:
teacher_forcing_ratio = 0.5
MAX_LENGTH=100
def evaluateGRU(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        encoder.eval()
        decoder.eval()
        #input_tensor = tensorFromSentence(input_lang, sentence)
        input_tensor = sentence.to(device)#datas['test_source.txt']
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            # if topi.item() == EOS_token:
            #     decoded_words.append('<EOS>')
            #     break
            # else:
            #     decoded_words.append(output_lang.index2word[topi.item()])
            if topi.item() == EOS_token:
                decoded_words.append(1)
                break
            else:
                decoded_words.append(topi.item())


            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

import random
def evaluateRandomly(encoder, decoder, n=1):#10):
    for i in range(n):
        ran = random.randrange(1,len(datas['test_source.txt']))
        pair = [datas['test_source.txt']['source'][ran], datas['test_target.txt']['target'][ran]]
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluateGRU(encoder, decoder, torch.Tensor(pair[0]).type(torch.long).unsqueeze(1))
        #output_sentence = ' '.join(output_words)
        print('<',output_words)
        #print('<', output_sentence)
        print('')

def corrects(encoder, decoder):

    corrects = 0
    for i in range(len(datas['test_source.txt'])):
        ran = random.randrange(1,len(datas['test_source.txt']))
        pair = [datas['test_source.txt']['source'][ran], datas['test_target.txt']['target'][ran]]
        output_words, attentions = evaluateGRU(encoder, decoder, torch.Tensor(pair[0]).type(torch.long).unsqueeze(1))
        if pair[1] == output_words:
            corrects+=1
    return corrects

def corrects_wide(encoder, decoder): 

    corrects = 0

    for i in range(len(datas['test_source.txt'])):
        ran = random.randrange(1,len(datas['test_source.txt']))
        pair = [datas['test_source.txt']['source'][ran], datas['test_target.txt']['target'][ran]]
        output_words, attentions = evaluateGRU(encoder, decoder, torch.Tensor(pair[0]).type(torch.long).unsqueeze(1))
        if pair[1] == output_words[:len(pair[1])]:
            corrects+=1
    return corrects


def corrects_vector(encoder, decoder): 

    corrects = 0
    accuracy = 0.0

    for i in range(len(datas['test_source.txt'])):
        ran = random.randrange(1,len(datas['test_source.txt']))
        pair = [datas['test_source.txt']['source'][ran], datas['test_target.txt']['target'][ran]]
        output_words, attentions = evaluateGRU(encoder, decoder, torch.Tensor(pair[0]).type(torch.long).unsqueeze(1))

        for k in range(min(len(pair[1]),len(output_words))):

            if pair[1][k] == output_words[k]:
                corrects+=1
        cur_acc = corrects/max(len(pair[1]),len(output_words))
        accuracy+=cur_acc
        corrects=0
    return accuracy/len(datas['test_source.txt'])



In [ ]:
vars = ['Adam','SGD']
hidden_size = 256
SOS_token = 0
EOS_token = 1
encoder1 = EncoderRNN(700, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, 700, dropout_p=0.1).to(device)
for var in vars:
    epath=''
    dpath=''
    for encoder_path in encoder_list:
        if var in encoder_path:
            encoder1.load_state_dict(torch.load(encoder_path))
            epath=encoder_path
            #print(encoder_path)
            break
    for decoder_path in decoder_list:
        if var in decoder_path:
            attn_decoder1.load_state_dict(torch.load(decoder_path))
            dpath = decoder_path
            break
    encoder1.eval()
    attn_decoder1.eval()
    acc_c_gru = corrects(encoder1,attn_decoder1 )
    acc_cw_gru = corrects_wide(encoder1,attn_decoder1)
    acc_v_gru = corrects_vector(encoder1,attn_decoder1)
    print(epath,dpath,"'s accuracy")
    print("",acc_c_gru/len(test_data_new))
    print("",acc_cw_gru/len(test_data_new))
    print("",acc_v_gru)